# **Base Model Prediction**

In this notebook, I'll perform the prediction with base model **This Notebook have prediction with Base Model.**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.linear_model import LinearRegression,Ridge
from sklearn import linear_model
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score,precision_recall_curve,mean_absolute_error
from sklearn.ensemble import VotingRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

Trying all of the above import models but data predicted well with Machine learning Model - **Linear Regression , Random Forest Regression , Gradient Boosting and Adaboost**

# **For 6 Hour**

## Load train data
Here, we'll load the train dataset.

In [4]:
# for 6 hour prediction

df = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/Model_1/train_preprocessed.csv')

Creating Fog_index_future column which need to be predicted

In [5]:
df["date_time"]=pd.to_datetime(df.date_time)
df.set_index("date_time",inplace=True)

df['fog_index_future']= df['fog_index'].shift(-1)

df.dropna(inplace=True)

Selecting winter month data for fog predcition , Here we are selecting the data of Nov, Dec, Jan and Feb month . By taking dataset from these four month, our data will become safe from unbalancing .

In [6]:
# selecting winter month(NDJF)
train =  df[(df.index.month==1) | (df.index.month==2)|(df.index.month==11) |(df.index.month==12)]

Creating X and y from train data

In [7]:
X_train,y_train = train.drop(["fog_index_future"],axis=1),train["fog_index_future"]

Similarly, creating X and y for Validation dataset

In [8]:
# taking validation dataset
val = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/Model_1/val_preprocessed.csv')

val["date_time"]=pd.to_datetime(val.date_time)
val.set_index("date_time",inplace=True)

val['fog_index_future']= val['fog_index'].shift(-1)

val.dropna(inplace=True)

# selecting winter month(NDJF)
validation =  val[(val.index.month==1) | (val.index.month==2)|(val.index.month==11) |(val.index.month==12)]

X_val,y_val = validation.drop(["fog_index_future"],axis=1),validation["fog_index_future"]


Standardising the data , because we are using linear regression model which works well when data is standardise .

In [9]:
stand_scaler = StandardScaler()
X_train_stand = stand_scaler.fit_transform(X_train)
X_val_stand = stand_scaler.transform(X_val)

Fitting RandomForestRegressor model

In [10]:
rf = RandomForestRegressor()

rf.fit(X_train,y_train)

print('RandomforestRegressor')
print(metrics.mean_squared_error(y_train,rf.predict(X_train)))
print(metrics.mean_squared_error(y_val,rf.predict(X_val)))

RandomforestRegressor
0.0016768322554841183
0.023176285514982756


Fitting LinearRegression model

In [11]:
lr = LinearRegression()


lr.fit(X_train_stand,y_train)

print('LinearRegressor')
print(metrics.mean_squared_error(y_train,lr.predict(X_train_stand)))
print(metrics.mean_squared_error(y_val,lr.predict(X_val_stand)))

LinearRegressor
0.01310605891981658
0.025232608601971237


Fitting GradientBoosting

In [12]:
gb =  GradientBoostingRegressor()
adb = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='friedman_mse',max_depth=15))
# ridge =  Ridge()
# sgd =  SGDRegressor()
# svr = SVR()
# knn = KNeighborsRegressor()


In [13]:
gb.fit(X_train,y_train)

print('Gradient Boosting')
print(metrics.mean_squared_error(y_train,gb.predict(X_train)))
print(metrics.mean_squared_error(y_val,gb.predict(X_val)))

Gradient Boosting
0.009571908949337576
0.02210636443349911


Fitting Adaboost

In [14]:
adb.fit(X_train,y_train)

print('Adaboost')
print(metrics.mean_squared_error(y_train,adb.predict(X_train)))
print(metrics.mean_squared_error(y_val,adb.predict(X_val)))

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Adaboost
0.000469492216477898
0.025217826016364697


Using Voting Ensemble

In [15]:
# voting ensemble
estimator = [('lr',lr),('rf',rf),('gb',gb),('adb',adb)]
vr = VotingRegressor(estimator)

vr.fit(X_train,y_train)

print('ensemble')
print(metrics.mean_squared_error(y_train,vr.predict(X_train)))
print(metrics.mean_squared_error(y_val,vr.predict(X_val)))

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


ensemble
0.0040427483376318145
0.022828537923611347


In these all of the used models RandomForestRegressor gives Minimum Mse value most of the time so it is chosen for plotting the graph

In [17]:
validation['pred']=  rf.predict(X_val)

In [19]:
# with previous data 6 hour prediction
import plotly.express as px
fig = px.line(validation.reset_index(),x='date_time',y=['fog_index_future','pred'])

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1,label='1y',step='year',stepmode='todate'),
            dict(count=2,label='2y',step='year',stepmode='todate'),
            dict(count=3,label='3y',step='year',stepmode='todate'),
            dict(step='all')
        ])
    )
)
fig.show()

# **For one Day**

In [20]:
# for 1 day

df = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/Model_1/train_preprocessed.csv')

df["date_time"]=pd.to_datetime(df.date_time)
df.set_index("date_time",inplace=True)

df['fog_index_future']= df['fog_index'].shift(-4)

df.dropna(inplace=True)

# selecting winter month(NDJF)
train =  df[(df.index.month==1) | (df.index.month==2)|(df.index.month==11) |(df.index.month==12)]


X_train,y_train = train.drop(["fog_index_future"],axis=1),train["fog_index_future"]

# taking validation dataset
val = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/Model_1/val_preprocessed.csv')

val["date_time"]=pd.to_datetime(val.date_time)
val.set_index("date_time",inplace=True)

val['fog_index_future']= val['fog_index'].shift(-4)


val.dropna(inplace=True)

# selecting winter month(NDJF)
validate =  val[(val.index.month==1) | (val.index.month==2)|(val.index.month==11) |(val.index.month==12)]


X_val,y_val = validate.drop(["fog_index_future"],axis=1),validate["fog_index_future"]


In [21]:
stand_scaler = StandardScaler()
X_train_stand = stand_scaler.fit_transform(X_train)
X_val_stand = stand_scaler.transform(X_val)

In [23]:
# running models
rf = RandomForestRegressor()


rf.fit(X_train,y_train)

print('RandomforestRegressor')
print(metrics.mean_squared_error(y_train,rf.predict(X_train)))
print(metrics.mean_squared_error(y_val,rf.predict(X_val)))

lr = LinearRegression()


lr.fit(X_train_stand,y_train)

print('LinearRegressor')
print(metrics.mean_squared_error(y_train,lr.predict(X_train_stand)))
print(metrics.mean_squared_error(y_val,lr.predict(X_val_stand)))

gb =  GradientBoostingRegressor()
adb = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='friedman_mse',max_depth=15))

gb.fit(X_train,y_train)

print('ensemble')
print(metrics.mean_squared_error(y_train,gb.predict(X_train)))
print(metrics.mean_squared_error(y_val,gb.predict(X_val)))

adb.fit(X_train,y_train)

print('ensemble')
print(metrics.mean_squared_error(y_train,adb.predict(X_train)))
print(metrics.mean_squared_error(y_val,adb.predict(X_val)))

RandomforestRegressor
0.0017953296761950097
0.02228819397160061
LinearRegressor
0.01201061659899603
0.020818037597610823
ensemble
0.009617873394902105
0.0208748708350662


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning:

`base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.



ensemble
0.0005305393831336776
0.026671763744181776


In [24]:
# voting ensemble
estimator = [('lr',lr),('rf',rf),('gb',gb),('adb',adb)]
vr = VotingRegressor(estimator)

vr.fit(X_train,y_train)

print('ensemble')
print(metrics.mean_squared_error(y_train,vr.predict(X_train)))
print(metrics.mean_squared_error(y_val,vr.predict(X_val)))

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning:

`base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.



ensemble
0.004043094730568731
0.021406907704646323


In [25]:
validation['pred']=  rf.predict(X_val)

In [26]:
# with previous data 6 hour prediction
import plotly.express as px
fig = px.line(validation.reset_index(),x='date_time',y=['fog_index_future','pred'])

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1,label='1y',step='year',stepmode='todate'),
            dict(count=2,label='2y',step='year',stepmode='todate'),
            dict(count=3,label='3y',step='year',stepmode='todate'),
            dict(step='all')
        ])
    )
)
fig.show()

# **For three Day**

In [27]:
# for 3 day

df = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/Model_1/train_preprocessed.csv')

df["date_time"]=pd.to_datetime(df.date_time)
df.set_index("date_time",inplace=True)

df['fog_index_future']= df['fog_index'].shift(-12)

df.dropna(inplace=True)

# selecting winter month(NDJF)
train =  df[(df.index.month==1) | (df.index.month==2)|(df.index.month==11) |(df.index.month==12)]


X_train,y_train = train.drop(["fog_index_future"],axis=1),train["fog_index_future"]

# taking validation dataset
val = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/Model_1/val_preprocessed.csv')

val["date_time"]=pd.to_datetime(val.date_time)
val.set_index("date_time",inplace=True)

val['fog_index_future']= val['fog_index'].shift(-12)


val.dropna(inplace=True)

# selecting winter month(NDJF)
validate =  val[(val.index.month==1) | (val.index.month==2)|(val.index.month==11) |(val.index.month==12)]


X_val,y_val = validate.drop(["fog_index_future"],axis=1),validate["fog_index_future"]


In [28]:
stand_scaler = StandardScaler()
X_train_stand = stand_scaler.fit_transform(X_train)
X_val_stand = stand_scaler.transform(X_val)

In [29]:
# running models
rf = RandomForestRegressor()


rf.fit(X_train,y_train)

print('RandomforestRegressor')
print(metrics.mean_squared_error(y_train,rf.predict(X_train)))
print(metrics.mean_squared_error(y_val,rf.predict(X_val)))

lr = LinearRegression()


lr.fit(X_train_stand,y_train)

print('LinearRegressor')
print(metrics.mean_squared_error(y_train,lr.predict(X_train_stand)))
print(metrics.mean_squared_error(y_val,lr.predict(X_val_stand)))

gb =  GradientBoostingRegressor()
adb = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='friedman_mse',max_depth=15))

gb.fit(X_train,y_train)

print('ensemble')
print(metrics.mean_squared_error(y_train,gb.predict(X_train)))
print(metrics.mean_squared_error(y_val,gb.predict(X_val)))

adb.fit(X_train,y_train)

print('ensemble')
print(metrics.mean_squared_error(y_train,adb.predict(X_train)))
print(metrics.mean_squared_error(y_val,adb.predict(X_val)))

RandomforestRegressor
0.0021747704518206745
0.0314588213725543
LinearRegressor
0.015350738256947716
0.029761850447665137
ensemble
0.012624931952466506
0.03054106811961365


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning:

`base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.



ensemble
0.000634896617474246
0.034237942173475394


In [30]:
# voting ensemble
estimator = [('lr',lr),('rf',rf),('gb',gb),('adb',adb)]
vr = VotingRegressor(estimator)

vr.fit(X_train,y_train)

print('ensemble')
print(metrics.mean_squared_error(y_train,vr.predict(X_train)))
print(metrics.mean_squared_error(y_val,vr.predict(X_val)))

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning:

`base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.



ensemble
0.005239392635105467
0.030656698281997602


In [31]:
validation['pred']=  rf.predict(X_val)

In [32]:
# with previous data 6 hour prediction
import plotly.express as px
fig = px.line(validation.reset_index(),x='date_time',y=['fog_index_future','pred'])

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1,label='1y',step='year',stepmode='todate'),
            dict(count=2,label='2y',step='year',stepmode='todate'),
            dict(count=3,label='3y',step='year',stepmode='todate'),
            dict(step='all')
        ])
    )
)
fig.show()

# **For five Day**

In [33]:
# for 5 day

df = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/Model_1/train_preprocessed.csv')

df["date_time"]=pd.to_datetime(df.date_time)
df.set_index("date_time",inplace=True)

df['fog_index_future']= df['fog_index'].shift(-20)

df.dropna(inplace=True)

# selecting winter month(NDJF)
train =  df[(df.index.month==1) | (df.index.month==2)|(df.index.month==11) |(df.index.month==12)]


X_train,y_train = train.drop(["fog_index_future"],axis=1),train["fog_index_future"]

# taking validation dataset
val = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/Model_1/val_preprocessed.csv')

val["date_time"]=pd.to_datetime(val.date_time)
val.set_index("date_time",inplace=True)

val['fog_index_future']= val['fog_index'].shift(-20)


val.dropna(inplace=True)

# selecting winter month(NDJF)
validate =  val[(val.index.month==1) | (val.index.month==2)|(val.index.month==11) |(val.index.month==12)]


X_val,y_val = validate.drop(["fog_index_future"],axis=1),validate["fog_index_future"]


In [34]:
stand_scaler = StandardScaler()
X_train_stand = stand_scaler.fit_transform(X_train)
X_val_stand = stand_scaler.transform(X_val)

In [35]:
# running models
rf = RandomForestRegressor()


rf.fit(X_train,y_train)

print('RandomforestRegressor')
print(metrics.mean_squared_error(y_train,rf.predict(X_train)))
print(metrics.mean_squared_error(y_val,rf.predict(X_val)))

lr = LinearRegression()


lr.fit(X_train_stand,y_train)

print('LinearRegressor')
print(metrics.mean_squared_error(y_train,lr.predict(X_train_stand)))
print(metrics.mean_squared_error(y_val,lr.predict(X_val_stand)))

gb =  GradientBoostingRegressor()
adb = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='friedman_mse',max_depth=15))
gb.fit(X_train,y_train)

print('ensemble')
print(metrics.mean_squared_error(y_train,gb.predict(X_train)))
print(metrics.mean_squared_error(y_val,gb.predict(X_val)))

adb.fit(X_train,y_train)

print('ensemble')
print(metrics.mean_squared_error(y_train,adb.predict(X_train)))
print(metrics.mean_squared_error(y_val,adb.predict(X_val)))

RandomforestRegressor
0.002342227308986258
0.03320118901864559
LinearRegressor
0.016023033097929244
0.0322815052889311
ensemble
0.01336734412095409
0.03282163395255064


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning:

`base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.



ensemble
0.0009769189577410803
0.035797685311924544


In [36]:
# voting ensemble
estimator = [('lr',lr),('rf',rf),('gb',gb),('adb',adb)]
vr = VotingRegressor(estimator)

vr.fit(X_train,y_train)

print('ensemble')
print(metrics.mean_squared_error(y_train,vr.predict(X_train)))
print(metrics.mean_squared_error(y_val,vr.predict(X_val)))

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning:

`base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.



ensemble
0.005675844725913549
0.03272942514537979


In [37]:
validation['pred']=  rf.predict(X_val)

In [38]:
# with previous data 6 hour prediction
import plotly.express as px
fig = px.line(validation.reset_index(),x='date_time',y=['fog_index_future','pred'])

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1,label='1y',step='year',stepmode='todate'),
            dict(count=2,label='2y',step='year',stepmode='todate'),
            dict(count=3,label='3y',step='year',stepmode='todate'),
            dict(step='all')
        ])
    )
)
fig.show()